In [15]:
import sys
print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, \
    outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale
import mne
import os
import numpy as np
import pandas as pd
from ieeg.calc.reshape import make_data_same
from ieeg.calc.stats import time_perm_cluster

from utils import *
import matplotlib.pyplot as plt

from pandas import read_csv
import scipy.stats as stats
import joblib

from scipy.ndimage import label

import json
import pickle

# rsa toolbox imports
from rsatoolbox.io.mne import read_epochs
from rsatoolbox.data.ops import merge_datasets
from os.path import join, expanduser, basename
import glob, json
import numpy, tqdm, mne, pandas
import rsatoolbox
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot

['C:\\Users\\jz421\\Desktop\\GlobalLocal\\IEEG_Pipelines', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\python311.zip', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\DLLs', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg', '', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32\\lib', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\Pythonwin', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\Pythonwin', 'C:/Users/j

In [16]:
subjects = ['D0057','D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103']
# load in subjects electrodes to rois dict. If it doesn't already exist, make it and then load it.
filename = 'subjects_electrodestoROIs_dict.json'
subjects_electrodestoROIs_dict = make_or_load_subjects_electrodes_to_rois_dict(filename, subjects)

Attempting to load the subjects' electrodes-to-ROIs dictionary...
Loaded data from subjects_electrodestoROIs_dict.json
Dictionary loaded successfully. Ready to proceed!


In [17]:
stimulus_locked = True  #toggle
response_locked = not stimulus_locked

if stimulus_locked:
    epochs_root_file = "Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8"
elif response_locked:
    epochs_root_file = "Response_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8"

In [18]:
stimulus_experiment_conditions = {
    "Stimulus_i25s25": {
        "BIDS_events": "Stimulus/i25.0/s25.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Stimulus_i25s75": {
        "BIDS_events": "Stimulus/i25.0/s75.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Stimulus_i75s25": {
        "BIDS_events": "Stimulus/i75.0/s25.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Stimulus_i75s75": {
        "BIDS_events": "Stimulus/i75.0/s75.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Stimulus_i25r25": {
        "BIDS_events": "Stimulus/i25.0/r25.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Stimulus_i25r75": {
        "BIDS_events": "Stimulus/i25.0/r75.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Stimulus_i75r25": {
        "BIDS_events": "Stimulus/i75.0/r25.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Stimulus_i75r75": {
        "BIDS_events": "Stimulus/i75.0/r75.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Stimulus_c25s25": {
        "BIDS_events": "Stimulus/c25.0/s25.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Stimulus_c25s75": {
        "BIDS_events": "Stimulus/c25.0/s75.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Stimulus_c75s25": {
        "BIDS_events": "Stimulus/c75.0/s25.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Stimulus_c75s75": {
        "BIDS_events": "Stimulus/c75.0/s75.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Stimulus_c25r25": {
        "BIDS_events": "Stimulus/c25.0/r25.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Stimulus_c25r75": {
        "BIDS_events": "Stimulus/c25.0/r75.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Stimulus_c75r25": {
        "BIDS_events": "Stimulus/c75.0/r25.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Stimulus_c75r75": {
        "BIDS_events": "Stimulus/c75.0/r75.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "75%"
    }
}

# congruency_conditions = {
#     "Stimulus_c": {
#         "BIDS_events": ["Stimulus/c25/s25", "Stimulus/c25/s75", "Stimulus/c75/s25", "Stimulus/c75/s75", "Stimulus/c25/r25", "Stimulus/c25/r75", "Stimulus/c75/r25", "Stimulus/c75/r75"],
#         "congruency": "c"
#     }
# }

stimulus_conditions = {
    "Stimulus_bigSsmallHtaskG": {
        "BIDS_events": "Stimulus/BigLetters/SmallLetterh/Taskg",
        "bigLetter": "s",
        "smallLetter": "h",
        "task": "g"
    },
    "Stimulus_bigSsmallHtaskL": {
        "BIDS_events": "Stimulus/BigLetters/SmallLetterh/Taskl",
        "bigLetter": "s",
        "smallLetter": "h",
        "task": "l"
    },
    "Stimulus_bigSsmallStaskG": {
        "BIDS_events": "Stimulus/BigLetters/SmallLetters/Taskg",
        "bigLetter": "s",
        "smallLetter": "s",
        "task": "g"
    },
    "Stimulus_bigSsmallStaskL": {
        "BIDS_events": "Stimulus/BigLetters/SmallLetters/Taskl",
        "bigLetter": "s",
        "smallLetter": "s",
        "task": "l"
    },
    "Stimulus_bigHsmallHtaskG": {
        "BIDS_events": "Stimulus/BigLetterh/SmallLetterh/Taskg",
        "bigLetter": "h",
        "smallLetter": "h",
        "task": "g"
    },
    "Stimulus_bigHsmallHtaskL": {
        "BIDS_events": "Stimulus/BigLetterh/SmallLetterh/Taskl",
        "bigLetter": "h",
        "smallLetter": "h",
        "task": "l"
    },
    "Stimulus_bigHsmallStaskG": {
        "BIDS_events": "Stimulus/BigLetterh/SmallLetters/Taskg",
        "bigLetter": "h",
        "smallLetter": "s",
        "task": "g"
    },
    "Stimulus_bigHsmallStaskL": {
        "BIDS_events": "Stimulus/BigLetterh/SmallLetters/Taskl",
        "bigLetter": "h",
        "smallLetter": "s",
        "task": "l"
    }
}

response_experiment_conditions = {
    "Response_i25s25": {
        "BIDS_events": "Response/i25.0/s25.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Response_i25s75": {
        "BIDS_events": "Response/i25.0/s75.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Response_i75s25": {
        "BIDS_events": "Response/i75.0/s25.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Response_i75s75": {
        "BIDS_events": "Response/i75.0/s75.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Response_i25r25": {
        "BIDS_events": "Response/i25.0/r25.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Response_i25r75": {
        "BIDS_events": "Response/i25.0/r75.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Response_i75r25": {
        "BIDS_events": "Response/i75.0/r25.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Response_i75r75": {
        "BIDS_events": "Response/i75.0/r75.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Response_c25s25": {
        "BIDS_events": "Response/c25.0/s25.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Response_c25s75": {
        "BIDS_events": "Response/c25.0/s75.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Response_c75s25": {
        "BIDS_events": "Response/c75.0/s25.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Response_c75s75": {
        "BIDS_events": "Response/c75.0/s75.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Response_c25r25": {
        "BIDS_events": "Response/c25.0/r25.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Response_c25r75": {
        "BIDS_events": "Response/c25.0/r75.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Response_c75r25": {
        "BIDS_events": "Response/c75.0/r25.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Response_c75r75": {
        "BIDS_events": "Response/c75.0/r75.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "75%"
    }
}

response_conditions = {
    "Response_bigSsmallHtaskG": {
        "BIDS_events": "Response/BigLetters/SmallLetterh/Taskg",
        "bigLetter": "s",
        "smallLetter": "h",
        "task": "g"
    },
    "Response_bigSsmallHtaskL": {
        "BIDS_events": "Response/BigLetters/SmallLetterh/Taskl",
        "bigLetter": "s",
        "smallLetter": "h",
        "task": "l"
    },
    "Response_bigSsmallStaskG": {
        "BIDS_events": "Response/BigLetters/SmallLetters/Taskg",
        "bigLetter": "s",
        "smallLetter": "s",
        "task": "g"
    },
    "Response_bigSsmallStaskL": {
        "BIDS_events": "Response/BigLetters/SmallLetters/Taskl",
        "bigLetter": "s",
        "smallLetter": "s",
        "task": "l"
    },
    "Response_bigHsmallHtaskG": {
        "BIDS_events": "Response/BigLetterh/SmallLetterh/Taskg",
        "bigLetter": "h",
        "smallLetter": "h",
        "task": "g"
    },
    "Response_bigHsmallHtaskL": {
        "BIDS_events": "Response/BigLetterh/SmallLetterh/Taskl",
        "bigLetter": "h",
        "smallLetter": "h",
        "task": "l"
    },
    "Response_bigHsmallStaskG": {
        "BIDS_events": "Response/BigLetterh/SmallLetters/Taskg",
        "bigLetter": "h",
        "smallLetter": "s",
        "task": "g"
    },
    "Response_bigHsmallStaskL": {
        "BIDS_events": "Response/BigLetterh/SmallLetters/Taskl",
        "bigLetter": "h",
        "smallLetter": "s",
        "task": "l"
    }
}

In [44]:
task='GlobalLocal'

if stimulus_locked:
    conditions = stimulus_conditions
elif response_locked:
    conditions = response_conditions #set this to whichever you're running

condition_names = list(conditions.keys()) # get the condition names as a list

# Assuming 'combined_data' is your DataFrame and 'subjects' is your list of subject IDs
subjects_mne_objects = create_subjects_mne_objects_dict(subjects=subjects, epochs_root_file=epochs_root_file, conditions=conditions, task="GlobalLocal", just_HG_ev1_rescaled=True, acc_trials_only=True)

Loading data for subject: D0057
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_HG_ev1_rescaled-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...    1500.00 ms
        0 CTF compensation matrices available
Not setting metadata
449 matching events found
No baseline correction applied
0 projection items activated
  Loading condition: Stimulus_bigSsmallHtaskG with parameters: {'BIDS_events': 'Stimulus/BigLetters/SmallLetterh/Taskg', 'bigLetter': 's', 'smallLetter': 'h', 'task': 'g'}
  Loading condition: Stimulus_bigSsmallHtaskL with parameters: {'BIDS_events': 'Stimulus/BigLetters/SmallLetterh/Taskl', 'bigLetter': 's', 'smallLetter': 'h', 'task': 'l'}
  Loading condition: Stimulus_bigSsmallStaskG with parameters: {'BIDS_events': 'Stimulus/BigLetters/SmallLetters/Taskg', 'bigLetter': 's', 'smallLetter': 's', 'task': 'g'}
  Loading condition: Stimulus_bi

In [45]:
# import mne
# import numpy as np
# from rsatoolbox.data import TemporalDataset

# def create_temporal_dataset(data_dict):
#     """
#     Create a TemporalDataset from a dictionary of Epochs data.
    
#     Parameters:
#     data_dict (dict): Dictionary with the structure subject -> condition -> HG_ev1_rescaled.
    
#     Returns:
#     TemporalDataset: The rsatoolbox TemporalDataset object created from the dictionary data.
#     """
#     # Initialize lists to hold the data and labels
#     data = []
#     obs_descriptors = {'subject': [], 'condition': []}
#     channel_descriptors = {}
#     time_descriptors = {}

#     # Extract the data
#     for subject, conditions in data_dict.items():
#         for condition, epochs_dict in conditions.items():
#             epochs = epochs_dict['HG_ev1_rescaled']
#             # Get the data from the Epochs object
#             epochs_data = epochs.get_data()  # shape: (n_epochs, n_channels, n_times)
            
#             # Append data and descriptors
#             data.append(epochs_data)
#             obs_descriptors['subject'].extend([subject] * epochs_data.shape[0])
#             obs_descriptors['condition'].extend([condition] * epochs_data.shape[0])

#     # Convert lists to numpy arrays
#     data = np.concatenate(data, axis=0)  # shape: (total_epochs, n_channels, n_times)

#     # Assuming the first Epochs object has the relevant info for channels and times
#     sample_epochs = next(iter(next(iter(data_dict.values())).values()))['HG_ev1_rescaled']
#     channel_descriptors['name'] = sample_epochs.ch_names
#     time_descriptors['times'] = sample_epochs.times

#     # Create TemporalDataset
#     dataset = TemporalDataset(
#         measurements=data,
#         obs_descriptors=obs_descriptors,
#         channel_descriptors=channel_descriptors,
#         time_descriptors=time_descriptors
#     )

#     return dataset

# # Example usage:
# dataset = create_temporal_dataset(subjects_mne_objects)
# print(dataset)


get significant channels

In [46]:
sig_chans_per_subject = get_sig_chans_per_subject(subjects, epochs_root_file, task='GlobalLocal', LAB_root=None)

Loaded significant channels for subject D0057
Loaded significant channels for subject D0059
Loaded significant channels for subject D0063
Loaded significant channels for subject D0065
Loaded significant channels for subject D0069
Loaded significant channels for subject D0071
Loaded significant channels for subject D0077
Loaded significant channels for subject D0090
Loaded significant channels for subject D0094
Loaded significant channels for subject D0100
Loaded significant channels for subject D0102
Loaded significant channels for subject D0103


In [47]:
# Assuming you have a list of subjects
subjects = ['D0057', 'D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103']
root_dir = rf"C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs"


In [48]:
# Define your ROIs dictionary and other parameters
rois_dict = {
    'lpfc': ["G_front_inf-Opercular", "G_front_inf-Orbital", "G_front_inf-Triangul", "G_front_middle", "G_front_sup", "Lat_Fis-ant-Horizont", "Lat_Fis-ant-Vertical", "S_circular_insula_ant", "S_circular_insula_sup", "S_front_inf", "S_front_middle", "S_front_sup"],
    'v1': ["G_oc-temp_med-Lingual", "S_calcarine", "G_cuneus"],
    'occ': ["G_cuneus", "G_and_S_occipital_inf", "G_occipital_middle", "G_occipital_sup", "G_oc-temp_lat-fusifor", "G_oc-temp_med-Lingual", "Pole_occipital", "S_calcarine", "S_oc_middle_and_Lunatus", "S_oc_sup_and_transversal", "S_occipital_ant"]
}

rois = list(rois_dict.keys())
# Assuming you have subjects_electrodestoROIs_dict and sig_chans_per_subject dictionaries
electrodes_per_subject_roi, sig_electrodes_per_subject_roi,  = make_sig_electrodes_per_subject_and_roi_dict(
    rois_dict, subjects_electrodestoROIs_dict, sig_chans_per_subject
)


For subject D0057, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['RAI6', 'RAI12', 'RAI13', 'RAI15', 'RAI16', 'RPI14', 'RAMF10', 'RAMF11', 'RAMF12', 'RAIF11', 'RAIF12', 'RAIF13', 'RAIF14']
For subject D0059, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['LMMF9', 'LMMF11', 'LMMF12', 'LPSF16']
For subject D0063, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['LOF16', 'LASF10', 'LASF14', 'LASF15', 'LASF16', 'LM

get electrode counts for each roi

In [49]:
total_electrodes_info = calculate_total_electrodes(sig_electrodes_per_subject_roi, electrodes_per_subject_roi)
for roi, counts in total_electrodes_info.items():
    print(f"Total number of {roi} electrodes across all subjects:", counts['total_electrodes'])
    print(f"Total number of significant {roi} electrodes across all subjects:", counts['total_significant_electrodes'])


Total number of lpfc electrodes across all subjects: 105
Total number of significant lpfc electrodes across all subjects: 46
Total number of v1 electrodes across all subjects: 17
Total number of significant v1 electrodes across all subjects: 12
Total number of occ electrodes across all subjects: 57
Total number of significant occ electrodes across all subjects: 41


check if any subjects have a weird sampling rate

In [50]:
# Assuming 'subjects_mne_objects' is your dictionary containing MNE objects for each subject
sampling_rate = 256
subject_rates = check_sampling_rates(subjects_mne_objects, expected_sampling_rate=sampling_rate)


Subject D0057 has the expected sampling rate: 256.0 Hz.
Subject D0059 has the expected sampling rate: 256.0 Hz.
Subject D0063 has the expected sampling rate: 256.0 Hz.
Subject D0065 has the expected sampling rate: 256.0 Hz.
Subject D0069 has the expected sampling rate: 256.0 Hz.
Subject D0071 has the expected sampling rate: 256.0 Hz.
Subject D0077 has the expected sampling rate: 256.0 Hz.
Subject D0090 has the expected sampling rate: 256.0 Hz.
Subject D0094 has the expected sampling rate: 256.0 Hz.
Subject D0100 has the expected sampling rate: 256.0 Hz.
Subject D0102 has the expected sampling rate: 256.0 Hz.
Subject D0103 has the expected sampling rate: 256.0 Hz.


create a dat dict like in the Temporal example from rsatoolbox 7/19  
#### Data Dictionary (dat) Structure
- **roi1**
  - **data**: 3D data array for this ROI
  - **channel_names**: List of significant channels for this ROI
  - **cond_names**: Dictionary where keys are condition names and values are integer indices
  - **cond_idx**: 1D array where each entry is an integer index corresponding to a cond_name
  - **times**: 1D array of times, corresponding to each sample
- **roi2**
  - **data**: 3D data array for this ROI
  - **channel_names**: List of significant channels for this ROI
  - **cond_names**: Dictionary where keys are condition names and values are integer indices
  - **cond_idx**: 1D array where each entry is an integer index corresponding to a cond_name
  - **times**: 1D array of times, corresponding to each sample
- **roiX**
  - **data**: 3D data array for this ROI
  - **channel_names**: List of significant channels for this ROI
  - **cond_names**: Dictionary where keys are condition names and values are integer indices
  - **cond_idx**: 1D array where each entry is an integer index corresponding to a cond_name
  - **times**: 1D array of times, corresponding to each sample


        




In [51]:
sig_electrodes = sig_electrodes_per_subject_roi['lpfc'].get('D0059', [])
sig_electrodes

['LMMF9', 'LMMF11', 'LMMF12', 'LPSF16']

In [63]:
import numpy as np

def prepare_data_for_temporal_dataset(subjects_mne_objects, condition_names, rois, subjects, sig_electrodes_per_subject_roi):
    # returns dat
    dat = {}
    overall_electrode_mapping = []
    electrode_mapping_per_roi = {roi: [] for roi in rois}  # Reinitialize for each processing run
    print('subjects: ', subjects)
    for roi in rois:
        dat[roi] = {}  # make a dict for each roi
        dat[roi]['channel_names'] = []  # initialize a list to hold channel names
        dat[roi]['condition_names'] = {}  # initialize dict where keys are condition names and values are integer indices
        dat[roi]['cond_idx'] = np.array([], dtype=int)  # initialize an empty 1D array for condition indices
        dat[roi]['times'] = np.array([])  # initialize an empty 1D array for time points

        # Determine all unique channels across subjects for this ROI, maintaining order
        all_channels = []
        for sub in subjects:
            sig_electrodes = sig_electrodes_per_subject_roi[roi].get(sub, [])
            sub_channel_names = [sub + '-' + sig_electrode for sig_electrode in sig_electrodes]
            for chan in sub_channel_names:
                if chan not in all_channels:
                    all_channels.append(chan)

        dat[roi]['channel_names'] = all_channels
        num_channels = len(all_channels)
        print('num channels: ', num_channels)

        # Initialize the data array with the number of trials and total channels
        dat[roi]['data'] = np.empty((0, num_channels, 0))  # initialize an empty 3D array for trials x channels x time points

        for sub in subjects:
            total_trials = 0  # Initialize counter for total trials across all conditions
            sig_electrodes = sig_electrodes_per_subject_roi[roi].get(sub, [])
            sub_channel_names = [sub + '-' + sig_electrode for sig_electrode in sig_electrodes]
            if not sig_electrodes:
                continue

            cond_idx = 0  # the example uses indexing from 1, but let's start from 0 because python
            for condition_name in condition_names:
                print(f'Processing {sub} for {condition_name} in {roi}')
                epochs = subjects_mne_objects[sub][condition_name]['HG_ev1_rescaled'].copy().pick(sig_electrodes)
                dat[roi]['condition_names'][condition_name] = cond_idx

                epochs_data = epochs.get_data(copy=True)
                num_trials, num_sub_channels, num_timepoints = epochs_data.shape

                print(f'Number of trials for {sub} in {condition_name}: {num_trials}')
                total_trials += num_trials

                # Initialize data array time dimension if it is empty
                if dat[roi]['data'].shape[2] == 0:
                    dat[roi]['data'] = np.empty((0, num_channels, num_timepoints))
                    dat[roi]['times'] = epochs.times

                # Create an array filled with NaNs for the current subject's data
                sub_data = np.full((num_trials, num_channels, num_timepoints), np.nan)

                # Find the indices for the subject's channels in the total list of channels
                channel_indices = [all_channels.index(chan) for chan in sub_channel_names]
                print('sub: ', sub)
                print("channel indices: ", channel_indices)
                
                # Place the subject's data in the correct indices
                sub_data[:, channel_indices, :] = epochs_data

                # Concatenate the new data along the first axis (trials)
                dat[roi]['data'] = np.concatenate((dat[roi]['data'], sub_data), axis=0)

                # Extend the cond_idx array
                dat[roi]['cond_idx'] = np.concatenate((dat[roi]['cond_idx'], np.full(num_trials, cond_idx)))

                cond_idx += 1  # increment cond_idx

            print(f'Total number of trials for {sub} across all conditions: {total_trials}')

    return dat

# Example call to the function (you need to replace the arguments with actual data)
dat = prepare_data_for_temporal_dataset(subjects_mne_objects, condition_names, rois, ['D0057', 'D0059'], sig_electrodes_per_subject_roi)


subjects:  ['D0057', 'D0059']
num channels:  6
Processing D0057 for Stimulus_bigSsmallHtaskG in lpfc
Number of trials for D0057 in Stimulus_bigSsmallHtaskG: 59
sub:  D0057
channel indices:  [0, 1]
Processing D0057 for Stimulus_bigSsmallHtaskL in lpfc
Number of trials for D0057 in Stimulus_bigSsmallHtaskL: 54
sub:  D0057
channel indices:  [0, 1]
Processing D0057 for Stimulus_bigSsmallStaskG in lpfc
Number of trials for D0057 in Stimulus_bigSsmallStaskG: 53
sub:  D0057
channel indices:  [0, 1]
Processing D0057 for Stimulus_bigSsmallStaskL in lpfc
Number of trials for D0057 in Stimulus_bigSsmallStaskL: 34
sub:  D0057
channel indices:  [0, 1]
Processing D0057 for Stimulus_bigHsmallHtaskG in lpfc
Number of trials for D0057 in Stimulus_bigHsmallHtaskG: 51
sub:  D0057
channel indices:  [0, 1]
Processing D0057 for Stimulus_bigHsmallHtaskL in lpfc
Number of trials for D0057 in Stimulus_bigHsmallHtaskL: 60
sub:  D0057
channel indices:  [0, 1]
Processing D0057 for Stimulus_bigHsmallStaskG in lpfc

In [64]:
405 + 427

832

In [75]:
dat['lpfc']['data'][405, , :]

IndexError: index 6 is out of bounds for axis 1 with size 6

In [13]:
def test():
    print('line 1')
    print('line 2')

test()

line 1
line 2
